In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
# There are multiple genres per movie
train_df = pd.read_csv("../dataset/train_data.csv",usecols=['genres', 'overview'], converters={"genres":literal_eval})
test_df = pd.read_csv("../dataset/test_data.csv",usecols=['genres', 'overview'], converters={"genres":literal_eval})
train_df.head()

,genres,overview
0,"[Comedy, Drama, Romance]",Beautiful chanteuse 'Bijou' (Marlene Dietrich)...
1,"[Action, Crime, Drama, Thriller]",In a post-apocalyptic world ravaged by feuding...
2,"[Drama, Romance, TV Movie, Western]","Marty is a 19 year old pioneer woman, recently..."
3,"[Action, Comedy]",A couple of fumbling best friends run a privat...
4,"[Comedy, Romance, TV Movie]",One woman's unexpected race to the altar teach...


In [3]:
# Initial train and test split.
from sklearn.model_selection import train_test_split

test_split = 0.1

train_df, val_df = train_test_split(
    train_df,
    test_size=test_split,
    stratify=train_df["genres"].values,
)

In [4]:
import tensorflow as tf


genres = tf.ragged.constant(train_df["genres"].values)
lookup = tf.keras.layers.StringLookup(output_mode="multi_hot", num_oov_indices=0)
lookup.adapt(genres)
vocab = lookup.get_vocabulary()

print("Vocabulary:\n")
print(vocab)

2023-06-19 20:49:35.558195: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:7622] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-06-19 20:49:35.558220: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-06-19 20:49:35.558229: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1497] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-19 20:49:35.563286: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-19 20:49:36.132048: W tensorflow/compiler/

Vocabulary:

['Drama', 'Comedy', 'Thriller', 'Romance', 'Action', 'Horror', 'Crime', 'Documentary', 'Adventure', 'Science Fiction', 'Family', 'Mystery', 'Fantasy', 'Animation', 'Music', 'Foreign', 'History', 'War', 'Western', 'TV Movie']


In [5]:
batch_size = 128

In [6]:
def make_dataset(dataframe, is_train=True):
    labels = tf.ragged.constant(dataframe["genres"].values)
    label_binarized = lookup(labels).numpy()
    dataset = tf.data.Dataset.from_tensor_slices(
        (dataframe["overview"].values, label_binarized)
    )
    dataset = dataset.shuffle(batch_size * 10) if is_train else dataset
    return dataset.batch(batch_size)


train_dataset = make_dataset(train_df, is_train=True)
validation_dataset = make_dataset(val_df, is_train=False)
test_dataset = make_dataset(test_df, is_train=False)

# Models

In [7]:
import keras
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping


def make_mlp_model(text_vectorizer):
    model = keras.Sequential(
        [
            tf.keras.Input(shape=(1,), dtype=tf.string, name='text'),
            text_vectorizer,
            layers.Dense(64, activation="relu"),
            layers.Dense(32, activation="relu"),
            layers.Dense(lookup.vocabulary_size(), activation="sigmoid"),
        ]
    )
    return model


def make_conv_model(text_vectorizer):
    model = keras.Sequential(
        [
            tf.keras.Input(shape=(1,), dtype=tf.string, name='text'),
            text_vectorizer,
            layers.Lambda(lambda x: tf.cast(x, "float32")),
            layers.Reshape((-1, 1)),
            layers.Conv1D(64, 3, activation='sigmoid'),
            layers.Conv1D(32, 3, activation='sigmoid'),
            layers.Flatten(),
            layers.Dense(lookup.vocabulary_size(), activation="sigmoid"),
        ]
    )
    return model


def make_lstm_model(text_vectorizer):
    model = tf.keras.Sequential([
        text_vectorizer,
        layers.Embedding(
            input_dim=len(text_vectorizer.get_vocabulary()),
            output_dim=64,
            mask_zero=True),
        layers.Bidirectional(tf.keras.layers.LSTM(64)),
        layers.Dense(64, activation='relu'),
        layers.Dense(lookup.vocabulary_size(), activation="sigmoid"),
    ])
    return model


def make_stacked_lstm_model(text_vectorizer):
    model = tf.keras.Sequential([
        text_vectorizer,
        tf.keras.layers.Embedding(len(text_vectorizer.get_vocabulary()), 64, mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        layers.Dense(lookup.vocabulary_size(), activation="sigmoid"),
    ])
    return model

# Util Functions

In [8]:
import matplotlib.pyplot as plt


def plot_result(history, item):
    plt.plot(history.history[item], label=item)
    plt.plot(history.history["val_" + item], label="val_" + item)
    plt.xlabel("Epochs")
    plt.ylabel(item)
    plt.title("Train and Validation {} Over Epochs".format(item), fontsize=14)
    plt.legend()
    plt.grid()
    plt.show()

# TF-IDF Vectorization with MLP Model

In [9]:
text_vectorizer = layers.TextVectorization(output_mode="tf_idf")
text_vectorizer.adapt(train_dataset.map(lambda text, label: text))

epochs = 2

model = make_mlp_model(text_vectorizer)

model.compile(
    loss="binary_crossentropy", 
    optimizer="adam",
    metrics=[tf.keras.metrics.BinaryAccuracy(), 
             tf.keras.metrics.CategoricalAccuracy(), 
             tf.keras.metrics.Accuracy(), 
             tf.keras.metrics.AUC(), 
             tf.keras.metrics.F1Score(average='macro'), 
             tf.keras.metrics.Precision(), 
             tf.keras.metrics.Recall()]
)

early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

history = model.fit(
    train_dataset, validation_data=validation_dataset, epochs=epochs, callbacks=[early_stopping_monitor], verbose=1
)

# plot_result(history, "loss")
# plot_result(history, "binary_accuracy")

model.evaluate(test_dataset)

Epoch 1/2


2023-06-19 20:49:53.004357: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fce701757e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-19 20:49:53.004374: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-06-19 20:49:53.007825: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-19 20:49:53.114598: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:434] Loaded cuDNN version 8600
2023-06-19 20:49:53.182574: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


262/262 [==============================] - 7s 18ms/step - loss: 0.2896 - binary_accuracy: 0.9020 - categorical_accuracy: 0.4939 - accuracy: 0.0000e+00 - auc: 0.8044 - f1_score: 0.1733 - precision: 0.5584 - recall: 0.2871 - val_loss: 0.2204 - val_binary_accuracy: 0.9202 - val_categorical_accuracy: 0.5423 - val_accuracy: 0.0000e+00 - val_auc: 0.8839 - val_f1_score: 0.2480 - val_precision: 0.7032 - val_recall: 0.3943
Epoch 2/2
33/33 [==============================] - 0s 14ms/step - loss: 0.2230 - binary_accuracy: 0.9191 - categorical_accuracy: 0.5584 - accuracy: 0.0000e+00 - auc: 0.8814 - f1_score: 0.2607 - precision: 0.6960 - recall: 0.3855


[0.22301112115383148,
 0.9191347360610962,
 0.558410108089447,
 0.0,
 0.8814190626144409,
 0.26068463921546936,
 0.6960389614105225,
 0.3854997754096985]

In [ ]:
max_seqlen = 150
text_vectorizer = layers.TextVectorization(output_sequence_length=max_seqlen, output_mode="int")
text_vectorizer.adapt(train_dataset.map(lambda text, label: text))

epochs = 2

model = make_conv_model(text_vectorizer)

model.compile(
    loss="binary_crossentropy", 
    optimizer="adam",
    metrics=[tf.keras.metrics.BinaryAccuracy(), 
             tf.keras.metrics.CategoricalAccuracy(), 
             tf.keras.metrics.Accuracy(), 
             tf.keras.metrics.AUC(), 
             tf.keras.metrics.F1Score(average='macro'), 
             tf.keras.metrics.Precision(), 
             tf.keras.metrics.Recall()]
)

early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

history = model.fit(
    train_dataset, validation_data=validation_dataset, epochs=epochs, callbacks=[early_stopping_monitor], verbose=1
)

# plot_result(history, "loss")
# plot_result(history, "binary_accuracy")

model.evaluate(test_dataset)

In [24]:
VOCAB_SIZE = 10000
text_vectorizer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
text_vectorizer.adapt(train_dataset.map(lambda text, label: text))

In [25]:
epochs = 2

model = make_lstm_model(text_vectorizer)

model.compile(
    loss="binary_crossentropy", 
    optimizer="adam",
    metrics=[tf.keras.metrics.BinaryAccuracy(), 
             tf.keras.metrics.CategoricalAccuracy(), 
             tf.keras.metrics.Accuracy(), 
             tf.keras.metrics.AUC(), 
             tf.keras.metrics.F1Score(average='macro'), 
             tf.keras.metrics.Precision(), 
             tf.keras.metrics.Recall()]
)

early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

history = model.fit(
    train_dataset, validation_data=validation_dataset, epochs=epochs, callbacks=[early_stopping_monitor], verbose=1
)

# plot_result(history, "loss")
# plot_result(history, "binary_accuracy")

model.evaluate(test_dataset)

Epoch 1/2


2023-06-19 20:11:39.090203: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_40/output/_23'
2023-06-19 20:11:39.599172: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:434] Loaded cuDNN version 8600
2023-06-19 20:11:39.899309: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f3ee807f720 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-19 20:

InternalError: Graph execution error:

Detected at node Adam/StatefulPartitionedCall_10 defined at (most recent call last):
  File "/home/ghaith/miniconda3/envs/movies_overview/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,

  File "/home/ghaith/miniconda3/envs/movies_overview/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,

  File "/home/ghaith/miniconda3/envs/movies_overview/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)

  File "/home/ghaith/miniconda3/envs/movies_overview/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,

  File "/home/ghaith/miniconda3/envs/movies_overview/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)

  File "/home/ghaith/miniconda3/envs/movies_overview/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.l

In [ ]:
epochs = 2

model = make_stacked_lstm_model(text_vectorizer)

model.compile(
    loss="binary_crossentropy", 
    optimizer="adam",
    metrics=[tf.keras.metrics.BinaryAccuracy(), 
             tf.keras.metrics.CategoricalAccuracy(), 
             tf.keras.metrics.Accuracy(), 
             tf.keras.metrics.AUC(), 
             tf.keras.metrics.F1Score(average='macro'), 
             tf.keras.metrics.Precision(), 
             tf.keras.metrics.Recall()]
)

early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

history = model.fit(
    train_dataset, validation_data=validation_dataset, epochs=epochs, callbacks=[early_stopping_monitor], verbose=1
)

# plot_result(history, "loss")
# plot_result(history, "binary_accuracy")

model.evaluate(test_dataset)

In [18]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

/home/ghaith/miniconda3/envs/movies_overview/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/ghaith/miniconda3/envs/movies_overview/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:37: UserWarning: You are currently using a nightly version of TensorFlow (2.14.0-dev20230619). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


In [19]:
from tfhub_maps import *

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [20]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [21]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_type_ids', 'input_mask', 'input_word_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [22]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [23]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.76268107  0.992807   -0.18603247  0.36670852  0.15221095  0.65519285
  0.9681294  -0.9486347   0.00215372 -0.98776615  0.06847879 -0.97630525]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.28957722  0.3430411   0.33181742 ...  0.2130085   0.71021616
  -0.05774301]
 [-0.28718334  0.3197859  -0.23049521 ...  0.5846513  -0.21335895
   0.72680455]
 [-0.6616913   0.6886561  -0.87458014 ...  0.10884289 -0.26172507
   0.47873232]
 ...
 [-0.22569078 -0.28944403 -0.07089932 ...  0.4759478   0.8325515
   0.4004193 ]
 [-0.2983998  -0.27478322 -0.05479976 ...  0.4884901   1.0953215
   0.18167159]
 [-0.44344297  0.00939324  0.07180209 ...  0.17316082  1.1833297
   0.07877684]]


In [24]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(lookup.vocabulary_size(), activation="sigmoid")(net)
  return tf.keras.Model(text_input, net)

In [25]:
epochs = 1
steps_per_epoch = tf.data.experimental.cardinality(train_dataset).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5

model = build_classifier_model()

optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

model.compile(
    loss="binary_crossentropy", 
    optimizer=optimizer,
    metrics=[tf.keras.metrics.BinaryAccuracy(), 
             tf.keras.metrics.CategoricalAccuracy(), 
             tf.keras.metrics.Accuracy(), 
             tf.keras.metrics.AUC(), 
             tf.keras.metrics.F1Score(average='macro'), 
             tf.keras.metrics.Precision(), 
             tf.keras.metrics.Recall()]
)

early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

history = model.fit(
    train_dataset, validation_data=validation_dataset, epochs=epochs, callbacks=[early_stopping_monitor], verbose=1
)

# plot_result(history, "loss")
# plot_result(history, "binary_accuracy")

model.evaluate(test_dataset)

33/33 [==============================] - 8s 237ms/step - loss: 0.2260 - binary_accuracy: 0.9124 - categorical_accuracy: 0.5371 - accuracy: 0.0000e+00 - auc_1: 0.8820 - f1_score: 0.1549 - precision_1: 0.6674 - recall_1: 0.3036


[0.22598876059055328,
 0.9124333262443542,
 0.5370818972587585,
 0.0,
 0.8819974064826965,
 0.15486115217208862,
 0.6673542857170105,
 0.3036133348941803]

In [62]:
import os

model_dir = "models/"
model_name = "model"
model_version = "1"
model_export_path = f"{model_dir}/{model_name}/{model_version}"

invert_stringlookup_layer = tf.keras.layers.StringLookup(vocabulary=vocab, invert=True)

model_for_inference = keras.Sequential([model, 
                                        layers.Lambda(lambda x: tf.round(x)),
                                        layers.Lambda(lambda x: tf.map_fn(lambda y: tf.where(y == 1.0)[..., 0] + 1, x, dtype=(tf.int64))),
                                        invert_stringlookup_layer
                                        ])

tf.saved_model.save(
    model_for_inference,
    export_dir=model_export_path,
)

print(f"SavedModel files: {os.listdir(model_export_path)}")

INFO:tensorflow:Assets written to: models//model/1/assets


INFO:tensorflow:Assets written to: models//model/1/assets


SavedModel files: ['variables', 'fingerprint.pb', 'assets', 'saved_model.pb']
